In [17]:
!pip install spotipy
!pip install python-dotenv

from pathlib import Path
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=Path('../.env'))

True

In [40]:
import numpy as np
import pandas as pd

In [18]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
# spotify credentials for the application:
# "Related Artist Network Visualizer"
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')
# create a credential manager and api layer
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Definimos el id del artista por el que empezaremos. En este caso, yo. 

In [22]:
jhook_id = '2vTS6RaNCyjiF0FkQDujjy'

In [23]:
name = sp.artist(jhook_id)

In [24]:
name

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2vTS6RaNCyjiF0FkQDujjy'},
 'followers': {'href': None, 'total': 104},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/2vTS6RaNCyjiF0FkQDujjy',
 'id': '2vTS6RaNCyjiF0FkQDujjy',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/cd8fbbec8379ce2b07d1f633297b5cef2c212587',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/143160e6e2579034840a1928d367b2bb48f64bce',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/5d2bd92bec0b5628f6905f7ec61c36776c3669c0',
   'width': 160}],
 'name': 'J Hook',
 'popularity': 1,
 'type': 'artist',
 'uri': 'spotify:artist:2vTS6RaNCyjiF0FkQDujjy'}

### Definimos las funciones que, recursivamente (en profundidad), generarán la red de artistas navegando por la API de Spotify.

In [53]:
def related_network(artist_id, depth=3):
    
    graph = dict()
    _related_network(artist_id, depth, graph)
    return graph

def _related_network(artist_id, depth, graph):
    if depth == 0:
        return
    name = sp.artist(artist_id)['name']
    like_artist = sp.artist_related_artists(artist_id)
    graph[name] = [related['name'] for related in like_artist['artists']]
    [_related_network(related['id'], depth - 1, graph) for related in like_artist['artists']]

In [26]:
# check the base case
related_network(jhook_id, 1)

{'J Hook': ['Shadejay', 'Califato ¾']}

In [31]:
# tomamos la red
network_artist = related_network(jhook_id, 4)

In [37]:
len(network_artist.keys())

207

#### Save the network so we can avoid generating it again

In [38]:
import json
with open('network_artist_4.json', 'w') as file:
    json.dump(network_artist, file)

In [47]:
def nodes(network_dict) -> pd.DataFrame:
    nodes = []

    # iterate over all the artists in the list
    for artist, related_list in network_dict.items():
        nodes.append(artist)
        nodes.extend(related_list)

    # keep only unique values to assign ids
    nodes = np.unique(nodes)
    # make a dataframe to generate ids
    nodes = pd.DataFrame(nodes, columns=['label'])
    # use the index columns as the id
    nodes['id'] = nodes.index
    return nodes

In [48]:
nodes(network_artist)

,label,id
0,$kyhook,0
1,091,1
2,3rajean,2
3,7 Notas 7 Colores,3
4,995,4
...,...,...
934,mori,934
935,shego,935
936,trashi,936
937,Ànteros,937


In [51]:
def edges(graph, nodes):
    '''Calculate all the conections between the artists'''
    
    edges = []

    for artist, related_list in graph.items():
        artist_node = nodes['id'][nodes['label'] == artist].values[0]
        for related in related_list:
            related_node = nodes['id'][nodes['label'] == related].values[0]
            edges.append((artist_node, related_node))

    return pd.DataFrame(edges, columns=['Source','Target'])

In [52]:
edges(network_artist, nodes(network_artist))

,Source,Target
0,369,784
1,369,100
2,100,853
3,100,187
4,100,486
...,...,...
4097,813,838
4098,813,419
4099,813,660
4100,813,754
